In [95]:
def get_files():
    #Edgelist of All Connected Nodes
    topcats = open('wiki-topcats.txt', 'r')

    #Page Names of Every Node
    topcats_page_names = open('wiki-topcats-page-names.txt', 'r')

    #Categories & All Pages Within Each Category
    topcats_categories = open('wiki-topcats-categories.txt', 'r')
    
    return topcats, topcats_page_names, topcats_categories

def load_data(topcats):
    data = np.loadtxt(topcats, dtype=int)
    return data

#Function to get all category names to a list
def all_categories(cat_list, lines):
    for line in lines:
        line = line.strip()
        line = line.split(' ')
        category = line[0]
        cat_list.append(category)
        
    return cat_list

#Function to convert categories (key) & pages in each category (value) into a dictionary
def lines_to_dict(dictionary, lines):
    for line in lines:
        line = line.strip()
        line = line.split(' ')
        key = line[0]
        values = line[1:]
        int_values = [eval(i) for i in values]
        dictionary[key] = int_values
        
    return dictionary

def make_catlist_catdict(topcats_categories):
    category_lines = topcats_categories.readlines()
    cat_list = []
    cat_list = all_categories(cat_list, category_lines)

    cat_dict = {}
    cat_dict = lines_to_dict(cat_dict, category_lines)
    
    return cat_list, cat_dict

#Only necessary if you want to see the page names/articles
def get_pages(topcats_page_names):
    page_lines = topcats_page_names.readlines()

    pages = {}

    for line in page_lines:
        line = line.strip()
        line = line.split(' ')
        key = int(line[0])
        values = line[1:]
        str_values = ' '.join(values)
        pages[key] = str_values
        
    return pages

def make_digraph(data):
    DG = nx.DiGraph()
    DG.add_edges_from(data)
    
    return DG

#Returns two random categories
def get_rand_cats(cat_list):
    cat_inds = []
    start = 0
    end = len(cat_list)

    for i in range(2):
        num = random.randint(start, end)
        cat_inds.append(num)
    
    #print(cat_inds)    
    return cat_inds

def get_rand_cat_list(num_cats, cat_list):
    cat_inds = []
    start = 0
    end = len(cat_list)

    for i in range(num_cats):
        num = random.randint(start, end)
        cat_inds.append(num)
    
    #print(cat_inds)    
    return cat_inds

#Returns Array of Page Numbers from Category Dictionary 
def get_cat_pages(category, cat_dict):
    return cat_dict.get(category)

def checks(data, cat_list, DG):
    if len(data) == 28511807:
        print('Data Transferred Correctly\n')
    else:
        print('Data Transferred Incorrectly\n')
        return False
    
    if len(cat_list) == 17364:
        print('Category List Transferred Correctly\n')
    else:
        print('Category List Transferred Incorrectly\n')
        return False
    
    if DG.number_of_nodes() == 1791489:
        print('DiGraph Made Correctly\n')
    else:
        print('DiGraph Made Incorrectly\n')
        return False
    
    print('All Check Passed')
    return True

#Path Functions

#Returns length of the shortest path between Node 1 and Node 2
def shortest_path_nodes(DG, node1, node2):
    path = nx.shortest_path(DG, node1, node2)
    return len(path)

#Returns Average Shortest Path Length of One Page from the First Category to Every Page From Another Category
def avg_shortest_path_nodes(DG, cat1_page, cat2):
    paths = []
    pages_in_cat2 = cat_dict.get(cat_list[cat2])
    
    for cat2_page in pages_in_cat2:
        path = shortest_path_nodes(DG, cat1_page, cat2_page)
        paths.append(path)
    
    np_paths = np.array(paths)
    avg_path_length = np.average(np_paths)
    
    return avg_path_length

#Returns the Average Shortest Path Length Between Two Categories
def avg_shortest_path_cats(DG, cat1, cat2):
    avg_paths = []
    pages_in_cat1 = cat_dict.get(cat_list[cat1])
    
    for cat1_page in pages_in_cat1:
        avg_path = avg_shortest_path_nodes(DG, cat1_page, cat2)
        avg_paths.append(avg_path)
    
    np_avg_paths = np.array(avg_paths)
    avg_path_cats = np.average(np_avg_paths)
    
    return avg_path_cats

#Correlation Functions

def find_correlations(DG, cat1, sub_cat_list):
    correlations = {}
    
    for cat2 in sub_cat_list:
        #Does no compare against itself
        if cat1 == cat2:
            continue
        
        #key_str = 'Category 1: ' + cat_list[cat1] + '-> Category 2: ' + cat_list[cat2]
        key_str = cat_list[cat1] + ' -> ' + cat_list[cat2]
        
        #Dictionary
        #every key is the input category, and then the category it is being compared to
        #the values is the avgerage shortest path between the two categories
        key = key_str
        values = avg_shortest_path_cats(DG, cat1, cat2)
        correlations[key] = values
    
    return correlations

def cat_correlations(DG, sub_cat_list):
    #array of dictionaries
    correlations = []
    
    for cat in sub_cat_list:
        correlations.append(find_correlations(DG, cat, sub_cat_list))   
    
    return correlations

#Min Correlation = Largset Number, meaning largest average shortest path between two categories
#Max Correlation = Smallest Number, meaning smallest avergae shortest path between two categories

def min_max_correlation(correlations, sub_cat_list):
    i = 0
    
    #Loops through array of dictionaries
    for correlation in correlations:
        #Initial Values
        #anything larger than 0 will replcae this number
        min_corr = 0 
        #anything smaller than 10 will replace this number, 10 chosen because the longest shortest path in the whole
        #dataset is 9
        max_corr = 10
        
        #Loops through dictionary to find min and max correlations
        for key in correlation:
            corr = correlation.get(key)
            if corr > min_corr:
                min_key = key
                min_corr = correlation.get(key)

            elif corr < max_corr:
                max_key = key
                max_corr = correlation.get(key)
            
        print(cat_list[sub_cat_list[i]])
        print(f'Minimum Correlation: {min_key}\nAverage Shortest Path Between Categories: {min_corr}')
        print(f'Maximum Correlatoin: {max_key}\nAverage Shortest Path Between Categories: {max_corr}\n')
        
        i += 1
        
def generate_sublist(subset_num, cat_list):
    sub_cat_list = get_rand_cat_list(subset_num, cat_list)
    for cat in sub_cat_list:
        print(cat, cat_list[cat])
        
    return sub_cat_list

#Examples

def print_path(DG, cat1, cat2, node1, node2, page_titles):
    path = nx.shortest_path(DG, node1, node2)
    path_str = ''
    step = 1
    print(f'\nStarting {cat1}')
    for i in path:
        page = page_titles.get(i)
        path_str = path_str + page + ' -> '
        print(f'Step: {step} - Page Title: {page}')
        step += 1
    print(f'Ending {cat2}')
    print(f'\nPath: {path_str[:-4]}')
    print(f'Path As Nodes: {path}')
    print(f'Length of Shortest Path: {len(path)}')

#Provides Base Examples of What this Project is About For Explanation Purposes
def example(DG, cat_list, cat_dict, page_titles):
    ex_cats = get_rand_cats(cat_list)
    ex_cat1 = cat_list[ex_cats[0]]
    ex_cat2 = cat_list[ex_cats[1]]
    print(f'Example Categories:\n{ex_cat1}\n{ex_cat2}')
    
    ex_cat1_pages = cat_dict.get(ex_cat1)
    ex_cat2_pages = cat_dict.get(ex_cat2)
    print(f'\nExample Category Pages:\n\n{ex_cat1}\nPages: {ex_cat1_pages}\n\n{ex_cat2}\nPages: {ex_cat2_pages}')
    
    ex_cat1_node = ex_cat1_pages[0]
    ex_cat1_node_page_title = page_titles.get(ex_cat1_node)
    ex_cat2_node = ex_cat2_pages[0]
    ex_cat2_node_page_title = page_titles.get(ex_cat2_node)
    print(f'\nExample Page/Article Titles:\n\n{ex_cat1}\nPage Node: {ex_cat1_node}\nPage Title: {ex_cat1_node_page_title}\n\n{ex_cat2}\nPage Node: {ex_cat2_node}\nPage Title: {ex_cat2_node_page_title}')
    
    print('\nExample Shortest Path:')
    print_path(DG, ex_cat1, ex_cat2, ex_cat1_node, ex_cat2_node, page_titles)
    ex_cat2_node2 = ex_cat2_pages[1]
    print_path(DG, ex_cat1, ex_cat2, ex_cat1_node, ex_cat2_node2, page_titles)
    
    print(f'\nExample Category List:')
    sublist_len5 = generate_sublist(5, cat_list)

In [79]:
#Start Up - will take a few minutes

#Imports
import pandas as pd
import numpy as np
import networkx as nx
import random

#Files
topcats, topcats_page_names, topcats_categories = get_files()
#Page Titles
page_titles = get_pages(topcats_page_names) 
#Data/Edge List
data = load_data(topcats)
#Category Name List & Category Page Dictionary
cat_list, cat_dict = make_catlist_catdict(topcats_categories)
#Make Graph
DG = make_digraph(data)
#Checks
checks(data, cat_list, DG)

Data Transferred Correctly

Category List Transferred Correctly

DiGraph Made Correctly

All Check Passed


True

In [36]:
subset_num = 5
sub_cat_list = get_rand_cat_list(subset_num, cat_list)
for cat in sub_cat_list:
    print(cat, cat_list[cat])

2428 Category:Canadian_ice_hockey_forwards;
5226 Category:United_States_Navy_nuclear_ships;
8650 Category:Rosids_of_Western_Australia;
5364 Category:Mammals_of_the_Northern_Territory;
5792 Category:Teachta_Dla;


In [37]:
correlations = cat_correlations(DG, sub_cat_list)
correlations

[{'Category:Canadian_ice_hockey_forwards; -> Category:United_States_Navy_nuclear_ships;': 6.066238100656124,
  'Category:Canadian_ice_hockey_forwards; -> Category:Rosids_of_Western_Australia;': 6.587187767669051,
  'Category:Canadian_ice_hockey_forwards; -> Category:Mammals_of_the_Northern_Territory;': 6.43020522770653,
  'Category:Canadian_ice_hockey_forwards; -> Category:Teachta_Dla;': 5.8135352855183555},
 {'Category:United_States_Navy_nuclear_ships; -> Category:Canadian_ice_hockey_forwards;': 6.218246292714378,
  'Category:United_States_Navy_nuclear_ships; -> Category:Rosids_of_Western_Australia;': 6.401094915722575,
  'Category:United_States_Navy_nuclear_ships; -> Category:Mammals_of_the_Northern_Territory;': 6.1533138768593485,
  'Category:United_States_Navy_nuclear_ships; -> Category:Teachta_Dla;': 5.919795236686905},
 {'Category:Rosids_of_Western_Australia; -> Category:Canadian_ice_hockey_forwards;': 6.867618121628817,
  'Category:Rosids_of_Western_Australia; -> Category:United

In [39]:
min_max_correlation(correlations, sub_cat_list)

Category:Canadian_ice_hockey_forwards;
Minimum Correlation: Category:Canadian_ice_hockey_forwards; -> Category:Rosids_of_Western_Australia;
Average Shortest Path Between Categories: 6.587187767669051
Maximum Correlatoin: Category:Canadian_ice_hockey_forwards; -> Category:Teachta_Dla;
Average Shortest Path Between Categories: 5.8135352855183555

Category:United_States_Navy_nuclear_ships;
Minimum Correlation: Category:United_States_Navy_nuclear_ships; -> Category:Rosids_of_Western_Australia;
Average Shortest Path Between Categories: 6.401094915722575
Maximum Correlatoin: Category:United_States_Navy_nuclear_ships; -> Category:Teachta_Dla;
Average Shortest Path Between Categories: 5.919795236686905

Category:Rosids_of_Western_Australia;
Minimum Correlation: Category:Rosids_of_Western_Australia; -> Category:Canadian_ice_hockey_forwards;
Average Shortest Path Between Categories: 6.867618121628817
Maximum Correlatoin: Category:Rosids_of_Western_Australia; -> Category:Mammals_of_the_Northern_T

In [96]:
example(DG, cat_list, cat_dict, page_titles)

Example Categories:
Category:Female_volleyball_players;
Category:Members_of_the_Order_of_the_British_Empire;

Example Category Pages:

Category:Female_volleyball_players;
Pages: [554082, 554084, 554085, 554086, 554087, 554088, 554089, 554090, 554091, 554092, 554093, 554094, 554217, 554226, 554227, 554228, 554229, 554230, 554231, 554232, 554233, 554248, 554250, 554262, 554270, 554271, 554272, 554273, 554277, 554282, 554369, 554370, 554371, 554372, 554373, 554374, 554375, 554376, 554377, 554378, 554379, 554383, 554384, 554385, 554386, 554387, 554389, 554390, 554391, 554396, 554397, 554398, 554401, 554411, 554412, 554413, 554414, 554417, 554425, 554430, 554431, 554432, 554438, 554459, 554463, 554464, 554465, 554466, 554467, 554468, 554469, 554470, 554471, 554475, 554485, 554487, 554488, 554489, 554496, 554497, 554499, 554500, 554501, 554505, 554506, 554510, 554511, 554512, 554513, 554519, 554520, 554521, 554522, 554523, 554524, 554526, 554527, 554530, 554531, 554532, 554533, 554534, 55453